# Session 2

<div style="text-align: justify">
In this second session we will apply the Perceptron algorithm to some classification tasks. A simple implementation of the Perceptron algorithm and its application is provided. The final purpose of this session is to apply the Perceptron algorithm to MyDigits dataset to train a matrix of weights that will be used in an application for Handwritten Digit Classification.
</div>

You may need to run this code if this is the first time you are running this notebook.

In [ ]:
!pip install seaborn scikit-learn pandas pillow gradio matplotlib

<p style="page-break-after:always;"></p>

# Perceptron

In [ ]:
#import warnings; warnings.filterwarnings("ignore")
import numpy as np

**Perceptron Classification:** classification of samples provided a weight matrix. Samples need to be prefixed by 1

$$c(\boldsymbol{x}) = \operatorname*{argmax}_{c} g_{c}(\boldsymbol{x})\text{, with }g_{c}(\boldsymbol{x})=\boldsymbol{w}_{c}^{t}\boldsymbol{x}\text{ for all }c$$

where $\boldsymbol{x} = (1, x_1, ..., x_D)^t$, $\mathbf{W} = (\boldsymbol{w}_1, \boldsymbol{w}_2, ...,  \boldsymbol{w}_C)$ and $\boldsymbol{w}_c = (w_{c0}, w_{c1}, ..., w_{cD})^t$

In [ ]:
def PerceptronClassification(X, W):
  Xh = np.hstack([np.ones((len(X), 1)), X])
  return np.argmax(Xh @ W, axis=1).reshape(-1, 1)

<p style="page-break-after:always;"></p>

**PerceptronTraining:** $\;$ Perceptron learns a matrix of weights $\mathbf{W}^*$ that minimizes the number of training errors (with margin $b$)
$$\mathbf{W}^*=\operatorname*{argmin}_{\mathbf{W}=(\boldsymbol{w}_1,\dotsc,\boldsymbol{w}_C)}\sum_n\;\mathbb{ I}\biggl(\max_{c\neq y_n}\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b \;>\; \boldsymbol{w}_{y_n}^t\boldsymbol{ x}_n\biggr)$$

It returns weights in homogeneous notation, $\mathbf{W}\in\mathbb{R}^{(1+D)\times C};\;$  together with the number of errors and iterations executed

> **Input:** $\;$ data $\;\mathcal{D}=\{(\boldsymbol{x}_n,y_n)\}\quad$ weights $\;\mathbf{W}=\{\boldsymbol{w}_c\}\quad$ learning rate $\;\alpha\in\mathbb{R}^{>0}\quad$ margin $\;b\in\mathbb{R}^{\geq 0}$ <br>
> **Output:** $\;$ optimized weights $\;\mathbf{W}^*=\{\boldsymbol{w}_c\}^*$ <br>
> `repeat` <br>
>> `for all` $\;$ training sample $\,\boldsymbol{x}_n$ <br>
>>> *err* = `False` <br>
>>> `for all` $\;$ class $\,c\neq y_n$ <br>
>>>> `if` $\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b>\boldsymbol{w}_{y_n}^t\boldsymbol{x}_n:\quad\boldsymbol{w}_c=\boldsymbol{w}_c-\alpha\boldsymbol{x}_n;\quad$ *err* = `True` <br>
>>>
>>> `if` $\;$ *err*: $\quad \boldsymbol{w}_{y_n}=\boldsymbol{w}_{y_n}+\alpha\boldsymbol{x}_n$
>
> `until` $\;$ no training sample is misclassified

In [ ]:
def PerceptronTraining(X, y, b=0.1, a=1.0, K=200):
    N, D = X.shape; Y = np.unique(y); C = Y.size; W = np.zeros((1+D, C))
    for k in range(1, K+1):  # for K iterations
        E = 0
        for n in range(N):  # for every training sample
            xn = np.array([1, *X[n, :]])
            cn = np.squeeze(np.where(Y==y[n]))  # Mapping to class labels from 0 to C-1 (for algorithmic simplicity)
            gn = W[:,cn].T @ xn; err = False
            for c in np.arange(C):  # for every class 
                if c != cn and W[:,c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a*xn; err = True
            if err:
                W[:, cn] = W[:, cn] + a*xn; E = E + 1
        if E == 0:
            break
    return W, E, k

<p style="page-break-after:always;"></p>

# Perceptron applied to the Iris dataset

**Reading the dataset:** $\;$ we also check that the data matrix and labels have the right number of rows and columns

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris(); X = iris.data.astype(np.float16)
y = iris.target.astype(np.uint).reshape(-1, 1)
print(X.shape, y.shape, "\n", np.hstack([X, y])[:5, :])

**Dataset partition:** $\;$ We create a split of the Iris dataset with $20\%$ of data for test and the rest for training, previously shuffling the data according to a given seed provided by a random number generator. Here, as in all code that includes randomness (which requires generating random numbers), it is convenient to fix said seed to be able to reproduce experiments with accuracy.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
print(X_train.shape, X_test.shape)

<p style="page-break-after:always;"></p>

**Learning a (linear) classifier with Perceptron:**

In [ ]:
W, E, k = PerceptronTraining(X_train, y_train)
print("Number of iterations executed: ", k)
print("Number of training errors: ", E)
print("Weight vectors of the classes (in columns and with homogeneous notation):\n", W)

**Calculation of test error rate:**

In [ ]:
y_test_pred = PerceptronClassification(X_test,W)
err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
print(f"Error rate on test: {err_test:.1%}")

<p style="page-break-after:always;"></p>

**Adjusting maximum number of iterations:**

In [ ]:
print(f"  alpha       b      K TrErr  TeErr")
print(f"------- ------- ------ ------ ------")
b = 0.1; a = 1.0
for K in (1, 2, 5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000):
    W, E, k = PerceptronTraining(X_train, y_train, b=b, a=a, K=K)
    y_test_pred = PerceptronClassification(X_test,W)
    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
    print(f"{a:.1e} {b:.1e} {k:6d} {E/len(X_train):6.1%} {err_test:6.1%}")

<p style="page-break-after:always;"></p>

**Adjusting the learning rate (alpha):**

In [ ]:
print(f"  alpha       b      K TrErr  TeErr")
print(f"------- ------- ------ ------ ------")
b = 0.1; K = 500
for a in (1e-3, 1e-2, 1e-1, 1e-0, 1e1, 1e2, 1e3):
    W, E, k = PerceptronTraining(X_train, y_train, b=b, a=a, K=K)
    y_test_pred = PerceptronClassification(X_test,W)
    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
    print(f"{a:.1e} {b:.1e} {k:6d} {E/len(X_train):6.1%} {err_test:6.1%}")

<p style="page-break-after:always;"></p>

**Adjusting the margin (b):**

In [ ]:
print(f"  alpha       b      K TrErr  TeErr")
print(f"------- ------- ------ ------ ------")
a = 1.0; K = 500
for b in (.0, .01, .1, 1, 10, 100):
    W, E, k = PerceptronTraining(X_train, y_train, b=b, a=a, K=K)
    y_test_pred = PerceptronClassification(X_test,W)
    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
    print(f"{a:.1e} {b:.1e} {k:6d} {E/len(X_train):6.1%} {err_test:6.1%}")

**Interpretation of results:** $\;$ the training data does not appear to be linearly separable; it is not clear that a margin greater than zero can improve results, especially since we only have $30$ test samples; with a margin $b=0.1$ we have already seen that an error (in test) of $16.7\%$ is obtained

<p style="page-break-after:always;"></p>

# Perceptron applied to the Digits dataset

**Reading the dataset:** $\;$ we also check that the data matrix and labels have the right number of rows and columns

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits(); X = digits.images.astype(np.float16).reshape(-1, 8*8); X/=np.max(X)
y = digits.target.astype(np.uint).reshape(-1, 1)
print(X.shape, y.shape, "\n", np.hstack([X, y])[:4, :])

<p style="page-break-after:always;"></p>

**Dataset partition:** $\;$ We create a split of the Iris dataset with $20\%$ of data for test and the rest for training, previously shuffling the data according to a given seed provided by a random number generator. Here, as in all code that includes randomness (which requires generating random numbers), it is convenient to fix said seed to be able to reproduce experiments with accuracy.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
print(X_train.shape, X_test.shape)

<p style="page-break-after:always;"></p>

**Adjusting maximum number of iterations:**

In [ ]:
print(f"  alpha       b      K TrErr  TeErr")
print(f"------- ------- ------ ------ ------")
b = 0.1; a = 1.0
for K in (1, 2, 5, 10, 20, 50, 100, 200):
    W, E, k = PerceptronTraining(X_train, y_train, b=b, a=a, K=K)
    y_test_pred = PerceptronClassification(X_test,W)
    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
    print(f"{a:.1e} {b:.1e} {k:6d} {E/len(X_train):6.1%} {err_test:6.1%}")

<p style="page-break-after:always;"></p>

**Adjusting the learning rate (alpha):**

In [ ]:
print(f"  alpha       b      K TrErr  TeErr")
print(f"------- ------- ------ ------ ------")
b = 0.1; K = 1000
for a in (1e-3, 1e-2, 1e-1, 1e-0, 1e1, 1e2, 1e3):
    W, E, k = PerceptronTraining(X_train, y_train, b=b, a=a, K=K)
    y_test_pred = PerceptronClassification(X_test,W)
    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
    print(f"{a:.1e} {b:.1e} {k:6d} {E/len(X_train):6.1%} {err_test:6.1%}")

<p style="page-break-after:always;"></p>

**Adjusting the margin (b):**

In [ ]:
print(f"  alpha       b      K TrErr  TeErr")
print(f"------- ------- ------ ------ ------")
a = 1e1; K = 1000
for b in (.0, .01, .1, 1, 10, 100):
    W, E, k = PerceptronTraining(X_train, y_train, b=b, a=a, K=K)
    y_test_pred = PerceptronClassification(X_test,W)
    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
    print(f"{a:.1e} {b:.1e} {k:6d} {E/len(X_train):6.1%} {err_test:6.1%}")

**Interpretation of results:** $\;$ the training data is linearly separable with training error equal to zero. In this case, it seems that small margins provide similar results on the test set with a lowest value of $3.6\%$.

<p style="page-break-after:always;"></p>

# Perceptron applied to MyDigits dataset

**Reading the dataset:**

In [ ]:
# Execute this cell only when running in Google Colab 
# You need to upload your images and labels files
from google.colab import files
uploaded = files.upload()

In [ ]:
from sklearn.model_selection import train_test_split

with open('images.npy', 'rb') as fd:
    X = np.load(fd)
    fd.close()

with open('labels.npy', 'rb') as fd:
    y = np.load(fd).astype(int)
    fd.close()

**Dataset partition:**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
print(X_train.shape, X_test.shape)

<p style="page-break-after:always;"></p>

**Adjusting maximum number of iterations:**

In [ ]:
print(f"  alpha       b      K TrErr  TeErr")
print(f"------- ------- ------ ------ ------")
# Complete

**Adjusting the learning rate (alpha):**

In [ ]:
print(f"  alpha       b      K TrErr  TeErr")
print(f"------- ------- ------ ------ ------")
# Complete

**Adjusting the margin (b):**

In [ ]:
print(f"  alpha       b      K TrErr  TeErr")
print(f"------- ------- ------ ------ ------")
# Complete

**Final classifier:** $\;$ Training final classifier with best parameters, saving and loading to test it 

In [ ]:
b = 0.1; a = 1.0; K = 200 # Replace with the best configuration obtained in the previous experiments
W, E, k = PerceptronTraining(X, y, b=b, a=a, K=K)
np.save("MyDigitsWeights.npy",W)

In [ ]:
with open('MyDigitsWeights.npy', 'rb') as fd:
    W = np.load(fd)
    fd.close()
y_test_pred = PerceptronClassification(X_test,W)
err_test = np.count_nonzero(y_test_pred.flatten() != y_test) / len(X_test)
print(f"Test error of final classifier: {err_test:.1%}")

In [ ]:
# Execute this cell only when running in Google Colab 
# You need to download MyDigitsWeights.npy
files.download('MyDigitsWeights.npy') 

<p style="page-break-after:always;"></p>

# Classify your own handwritten digits

<p style="text-align: justify">The following simple application allows you to classify your own handwritten digits. When you run this application, it shows a basic graphical interface containing a panel on which you can draw your own handwritten digits.</p>

<p style="text-align: justify">Before you can draw a digit, you need to click on the *pen* locate on the left vertical. Then you can draw on the panel. If you need to erase what you have drawn on the panel, just click on *bin* located on the top menu.</p>

<p style="text-align: justify">You can classify the image on the panel by clicking on the bottom bar labeled with *Classify image".</p>

In [ ]:
# Execute this cell only when running in Google Colab 
# You need to upload DigitClassifyGradioApp.py
from google.colab import files
uploaded = files.upload()

In [ ]:
from DigitClassifyGradioApp import create_interface

fn = input("Please provide filename for weight matrix:")
with open(fn, 'rb') as fd:
    W = np.load(fd)
    fd.close()

demo = create_interface(W, PerceptronClassification)
demo.launch()